# Extended Kalman Filter sample question

1. linearized motion model

2. linearized measured model

3. prediction

4. optimal gain

5. correction

In [13]:
import numpy as np

class EKFSample():
  def __init__(self):
    self.x_0 = np.array([[0,5]], dtype=np.float).T
    self.delta_t = 0.5
    self.u = -2.
    self.y_1 = np.pi/6
    self.S = 20
    self.D = 40
    self.p_check = 0.0
    self.P_check = np.array([[0.01,0],[0,1]], dtype=np.float)
    # remind that x = [[p], [p_dot]] so p_check is x[0][0]

    self.Jacobian = {
        'F' : np.array([[1, self.delta_t],[0,1]], dtype=np.float),
        'L': np.array([[1,0],[0,1]]),
        'H' : self.Jacobian_H(self.p_check),
        'M' : np.array([[1]]),
        'Q' : np.array([[0.1,0],[0,0.1]]),
        'R' : np.array([[0.01]])
    }

  def Jacobian_H(self, p_check):
    arctan_compute = self.S/((self.D - p_check)**2 + self.S**2)
    H = np.array([arctan_compute, 0], dtype=np.float)
    print(f"H matrix is {H}")

    return H

  def _update_p(self, new_p, new_P):
    self.p_check = new_p
    self.P_check = new_P

  def prediction(self, x_hat, w=0):
    x_check = self._f(x_hat, self.u)
    
    p_check = x_check[0][0]
    p_dot_check = x_check[1][0]

    P_check = self.Jacobian['F'].dot(self.P_check).dot(self.Jacobian['F'].T) + self.Jacobian['L'].dot(self.Jacobian['Q']).dot(self.Jacobian['L'].T)
    print(f"p_check : {p_check}, p_dot_check : {p_dot_check}, P_check : {P_check}")

    self._update_p(p_check, P_check)
    self.Jacobian['H'] = self.Jacobian_H(self.p_check) # update for every p update since init function is called only once :(

    return x_check
  
  def _f(self, x_k_1, u_k_1, w_k_1 = 0):
    ans = np.array([[1, self.delta_t], [0, 1]]).dot(x_k_1) + np.array([[0, self.delta_t]]).T*self.u
    return ans

  def _h(self, x_check, v =0):
    ans = np.arctan(self.S/(self.D - x_check[0][0]))
    return ans

  def correction(self, x_check):
    K_1 = self.Jacobian['H'].dot(self.P_check).dot(self.Jacobian['H'].T)
    # print(f"H : {self.Jacobian['H']}, P_check :  {self.P_check}, K_1 : {K_1}")
    K_2 = self.Jacobian['M'].dot(self.Jacobian['R']).dot(self.Jacobian['M'].T)
    # print(f"M : {self.Jacobian['M']}, R : {self.Jacobian['R']}, K_2 : {K_2}")

    K_3 = np.linalg.inv(K_1 + K_2)
    K = self.P_check.dot(self.Jacobian['H'].T)*(K_3)
    # print(f"K_1 : {K_1}, K_2 : {K_2}, K_3 : {K_3}")
    # print(f"P_check : {self.P_check}, H : {self.Jacobian['H']}, K : {K}")

    h_fn = self._h(x_check)
    print(f"h_fn : {h_fn}")
    print(f"Y_1 : {self.y_1}")

    x_hat = x_check + K.T*(self.y_1 - h_fn)
    return K, x_hat
    
if __name__ == "__main__":
  # point 1 : _h fn and _f fn are TOTALLY different!
  ekf = EKFSample()
  x_0 = np.array([[0,5]]).T
  x_check = ekf.prediction(x_0)
  print(f"x_check is : {x_check}")

  K, x_hat = ekf.correction(x_check)
  print(f"K is : {K}")
  print(f"x_hat is : {x_hat}")


H matrix is [0.01 0.  ]
p_check : 2.5, p_dot_check : 4.0, P_check : [[0.36 0.5 ]
 [0.5  1.1 ]]
H matrix is [0.01107266 0.        ]
x_check is : [[2.5]
 [4. ]]
h_fn : 0.4899573262537283
Y_1 : 0.5235987755982988
K is : [[0.39686426 0.55120036]]
x_hat is : [[2.51335109]
 [4.01854318]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: `np.floa